<a href="https://colab.research.google.com/github/lucasvazelle/SORBONNE_DATA_CHALLENGE_SALT/blob/main/SDC_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers sentencepiece joblib

In [2]:
# === 1. IMPORTS ===
import pandas as pd
import os
import re
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from lightgbm import LGBMClassifier
from collections import Counter, defaultdict
import joblib

from google.colab import drive
drive.mount('/content/drive')

# === 2. PARAMÈTRES ===
TRAIN_GRAPH_DIR = "/content/drive/MyDrive/SDC/Fichiers source/folder_training_set/folder_training_set"
TEST_GRAPH_DIR = "/content/drive/MyDrive/SDC/Fichiers source/folder_test_set/folder_test_set"
TRAIN_CSV = "/content/drive/MyDrive/SDC/Fichiers source/training_set_metadata.csv"
TEST_CSV = "/content/drive/MyDrive/SDC/Fichiers source/test_set_metadata_to_predict.xlsx"
OUTPUT_DIR = "/content/drive/MyDrive/SDC/Fichiers source/Output"
OUTPUT_CSV = os.path.join(OUTPUT_DIR, "submission_tf_idf_features_lgbopti.xlsx")
PICKLE_PATH = os.path.join(OUTPUT_DIR, "train_preprocessed_features_lgb_opti.pkl")
MODEL_PATH = os.path.join(OUTPUT_DIR, "model_lgbm_opti_features.pkl")
SAMPLE_SIZE = 13000



Mounted at /content/drive


In [ ]:
# === 3. FONCTIONS D'EXTRACTION ===
def entropy(vals):
    probs = np.array(list(vals.values())) / sum(vals.values())
    return -np.sum(probs * np.log2(probs + 1e-9))

def extract_features(filepath):
    with open(filepath, 'r') as f:
        content = f.read()

    inst_matches = re.findall(r'INST\s+:\s+([^:\n]+)', content)
    all_text = " ".join(inst_matches)
    c = Counter([w.lower() for w in inst_matches])

    top_instrs = ['call','jmp','ret','mov','push','pop','lea','cmp','xor','add','sub','test','and','or','inc','dec','nop','je','jne','jg','jl','jz','jge','jle','js','jns','jb','ja','jbe','jae']
    stats = {f'instr_{k}': c.get(k, 0) for k in top_instrs}

    stats['nb_tokens'] = len(inst_matches)
    stats['unique_instructions'] = len(set(inst_matches))
    stats['instruction_entropy'] = entropy(c)
    stats['ratio_jmp_call'] = c.get('jmp', 0) / (c.get('call', 1))

    # Graph structure
    edges = re.findall(r'"([0-9a-fx]+)"\s*->\s*"([0-9a-fx]+)"', content)
    nodes = set()
    graph = defaultdict(list)
    out_degree = defaultdict(int)
    for src, dst in edges:
        nodes.add(src)
        nodes.add(dst)
        out_degree[src] += 1
        graph[src].append(dst)

    stats['nb_nodes'] = len(nodes)
    stats['nb_edges'] = len(edges)
    stats['avg_out_degree'] = np.mean(list(out_degree.values())) if out_degree else 0
    stats['nb_terminal_nodes'] = sum(1 for node in nodes if out_degree[node] == 0)
    stats['nb_branches'] = sum(1 for node in out_degree if out_degree[node] > 1)
    stats['graph_density'] = stats['nb_edges'] / (stats['nb_nodes'] * (stats['nb_nodes']-1) + 1e-6) if stats['nb_nodes'] > 1 else 0

    def dfs_iterative(start_node):
        visited = set()
        stack = [(start_node, 0)]
        max_depth = 0
        while stack:
            node, depth = stack.pop()
            if node not in visited:
                visited.add(node)
                max_depth = max(max_depth, depth)
                for child in graph.get(node, []):
                    if child not in visited:
                        stack.append((child, depth + 1))
        return max_depth

    entry_node = list(nodes)[0] if nodes else None
    stats['depth_max'] = dfs_iterative(entry_node) if entry_node else 0

    return all_text, stats

def process_graphs(folder, selected_ids):
    texts, stats = {}, {}
    for fname in tqdm(os.listdir(folder)):
        if fname.endswith('.json'):
            id_ = fname.replace('.json', '')
            if id_ in selected_ids:
                path = os.path.join(folder, fname)
                txt, feat = extract_features(path)
                texts[id_] = txt
                stats[id_] = feat
    return texts, stats


In [ ]:

# === 4. PRÉTRAITEMENT ===
train_labels = pd.read_csv(TRAIN_CSV, sep=None, engine='python')
train_labels['name'] = train_labels['name'].astype(str)
sampled = train_labels.sample(n=SAMPLE_SIZE, random_state=42)
selected_ids = set(sampled['name'])
train_texts, train_stats = process_graphs(TRAIN_GRAPH_DIR, selected_ids)

# Fusion DataFrames
X_text = pd.Series(train_texts).rename_axis('name').reset_index(name='text')
X_stats = pd.DataFrame.from_dict(train_stats, orient='index').reset_index().rename(columns={'index': 'name'})
df = pd.merge(X_text, X_stats, on='name')
df = pd.merge(df, sampled, on='name')

# TF-IDF + SVD
vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=30000)
X_tfidf = vectorizer.fit_transform(df['text'])
svd = TruncatedSVD(n_components=3000)
X_svd = svd.fit_transform(X_tfidf)
explained_var = np.cumsum(svd.explained_variance_ratio_)
n_comp_opt = np.argmax(explained_var >= 0.85) + 1
svd = TruncatedSVD(n_components=n_comp_opt)
X_svd = svd.fit_transform(X_tfidf)

# Fusion
feature_cols = [col for col in X_stats.columns if col != 'name']
X_full = np.concatenate([X_svd, df[feature_cols].values], axis=1)
y = df.drop(columns=['name', 'text'] + feature_cols).astype(int)
X_tr, X_val, y_tr, y_val = train_test_split(X_full, y, test_size=0.2, random_state=42)


100%|██████████| 13185/13185 [06:06<00:00, 35.98it/s] 


In [ ]:
# === 5. ENTRAÎNEMENT LGBM SANS GRIDSEARCH, AVEC is_unbalance ===
clf = MultiOutputClassifier(
    LGBMClassifier(
        n_jobs=-1,
        is_unbalance=True,   # pour gérer les classes rares
        random_state=42
    )
)
clf.fit(X_tr, y_tr)



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 10, number of negative: 5868
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003429 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26513
[LightGBM] [Info] Number of data points in the train set: 5878, number of used features: 104
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001701 -> initscore=-6.374684
[LightGBM] [Info] Start training from score -6.374684
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 416, number of negative: 5462
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003177 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26513
[LightGBM] [Info] Number of data points in the train set: 5878, number of used features: 104
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.070772 -> initscore=-2.574885
[LightGBM] [Info] Start training from score -2.574885


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Number of positive: 1, number of negative: 5877
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26513
[LightGBM] [Info] Number of data points in the train set: 5878, number of used features: 104
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000170 -> initscore=-8.678802
[LightGBM] [Info] Start training from score -8.678802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 34, number of negative: 5844
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002788 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26513
[LightGBM] [Info] Number of data points in the train set: 5878, number of used features: 104
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.005784 -> initscore=-5.146810
[LightGBM] [Info] Start training from score -5.146810
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


MultiOutputClassifier(estimator=LGBMClassifier(is_unbalance=True, n_jobs=-1,
                                               random_state=42))

In [ ]:
# === 6. ÉVALUATION ===
y_val_pred = clf.predict(X_val)



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

In [ ]:
print("F1 Macro:", f1_score(y_val, y_val_pred, average='macro', zero_division=0))

F1 Macro: 0.22318894902925981


In [ ]:

# === 7. SAUVEGARDES ===
joblib.dump((X_full, y), PICKLE_PATH)
joblib.dump(clf, MODEL_PATH)
joblib.dump(vectorizer, os.path.join(OUTPUT_DIR, 'vectorizer.pkl'))
joblib.dump(svd, os.path.join(OUTPUT_DIR, 'svd.pkl'))



['/content/drive/MyDrive/SDC/Fichiers source/Output/svd.pkl']

In [ ]:
# === 8. TEST SET & EXPORT ===
test_meta = pd.read_excel(TEST_CSV)
test_ids = set(test_meta['name'].astype(str))
test_texts, test_stats = process_graphs(TEST_GRAPH_DIR, test_ids)

X_test_text = pd.Series(test_texts).rename_axis('name').reset_index(name='text')
X_test_stats = pd.DataFrame.from_dict(test_stats, orient='index').reset_index().rename(columns={'index': 'name'})
df_test = pd.merge(X_test_text, X_test_stats, on='name')

X_test_tfidf = vectorizer.transform(df_test['text'])
X_test_svd = svd.transform(X_test_tfidf)
X_test_stats_final = df_test[feature_cols].fillna(0)
X_test_final = np.concatenate([X_test_svd, X_test_stats_final], axis=1)



100%|██████████| 3000/3000 [28:18<00:00,  1.77it/s]


In [ ]:

# === 9. PREDICTIONS + EXPORT ===
y_test_pred = clf.predict(X_test_final)
df_preds = pd.DataFrame(y_test_pred, columns=y.columns)
df_preds.insert(0, 'name', df_test['name'])
final_submission = pd.merge(test_meta[['name']], df_preds, on='name', how='left')
final_submission.fillna(0, inplace=True)
final_submission.iloc[:, 1:] = final_submission.iloc[:, 1:].astype(int)
final_submission.to_excel(OUTPUT_CSV, index=False)
print(f"✅ Soumission sauvegardée dans : {OUTPUT_CSV}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

✅ Soumission sauvegardée dans : /content/drive/MyDrive/SDC/Fichiers source/Output/submission_tf_idf_features_lgbopti.xlsx
